In [77]:
met_en_zonder = {}
met_pr = {}
zonder_pr = {}
for T in searchobs:
    ob = searchobs[T].matched_text
    pr = [s for s in ob.spans if s.type == 'president']
    if len(pr)==0:
        zonder_pr[T]=pr
    else:
        met_pr[T]=pr
met_en_zonder['met_pr'] = met_pr
met_en_zonder['zonder_pr'] = zonder_pr

The match and span object takes the unmarked text from the objects and tries to match them with a delegate using the code for identifying delegates. But the failures in the ocr complicates this so that we have to use isolated strings, instead of splitting the text by commas or periods and newlines. This  drops all sorts of prefixes and in the case of a composite name such as _Taats van Amerongen_ it also either makes a name match twice or match part of it with another name. Also the prefixes like _van_ can get matched to the wrong names. So what we will do is match the names and then return to the list of spans and see if we cannot make a better match with the whole names. We would also like  to include previously acquired knowledge for better matching, but we have to think about that more.
Another thing is that we may overfit existing names, but we'll see about that.

In [80]:
config = {'char_match_threshold': 0.7,
 'ngram_threshold': 0.6,
 'levenshtein_threshold': 0.6,
 'ignorecase': False,
 'ngram_size': 2,
 'skip_size': 2}

sps = FuzzyKeywordSearcher(config=config)
spns = [(s.pattern, s.get_delegate()) for s in ob.spans if s]
for s in spns:
    nm = s[1]['name']
    d_id = s
#    if nm == 'Taets van Amerongen':
    if sps.keyword_index.get(nm):
        sps.index_keywords(nm)

In [81]:
spns

[('', {'id': 0, 'name': '', 'score': 0}),
 ('', {'id': 0, 'name': '', 'score': 0}),
 ('', {'id': 0, 'name': '', 'score': 0}),
 ('', {'id': 0, 'name': '', 'score': 0})]

In [82]:
matchsearch = FuzzyKeywordSearcher(config=fuzzysearch_config)
kws = defaultdict(list)
matcher = {}
for key in all_matched:
    variants = all_matched[key].get('variants')
    keyword = all_matched[key].get('m_kw')
    idnr = all_matched[key].get('id')
    name = all_matched[key].get('name')
    for variant in variants:
        kws[keyword].append(variant[0])
        matcher[variant[0]] = {'kw':keyword, 'id':idnr, 'name':name}
matchsearch.index_keywords(list(kws.keys()))
for k in kws:
    for v in kws[k]:
        matchsearch.index_spelling_variant(keyword=k,variant=v)

In [207]:
deputyregister = defaultdict(list)
search_results = {}
fm = FndMatch(year=1728,
                 #patterns=pats,
                 register=register,
                 rev_graph=transposed_graph,
                 searcher=herensearcher,
                 junksearcher=junksweeper,
                 df=abbreviated_delegates)
for T in searchobs:
    ob = searchobs[T].matched_text
    mo = MatchAndSpan(ob, junksweeper=junksweeper, previously_matched=previously_matched, match_search=matchsearch)
    for span in ob.spans:
        if span.type in ['president', 'delegate']:
            if span.pattern == '':
                span.set_pattern(pattern=ob.item[span.begin:span.end])
            r = fm.match_candidates(heer=span.pattern)
            span.set_delegate(delegate_id=r.id, 
                              delegate_name=r.proposed_delegate, 
                              delegate_score=r.score)

In [93]:
matchfnd

## Continue matching

In [204]:
defo = defaultdict(list)
for item in ob.get_unmatched_text():
    item = [i.strip() for i in re.split('[\.|,]', item)]
    for i in item:
        identified = ''
        res = {}
        r2 = {}
        for g in moregentlemen:
            mn = 0
            co = 0
            mc = 0
            vrts = g.variants['general']
            dr = [score_levenshtein_distance_ratio(x.form, i) for x in vrts if x.heerid==g.heerid]
            if len(dr)>0:
                mn = mean(dr)
            res[g] = {'mn':mn}
            if len(i)<5:
                co = [score_ngram_overlap_ratio(x.form, i, 1) for x in vrts if x.heerid==g.heerid]
                if co and len(co)>0:
                  mc = mean(co)
            res[g]['mc'] = mc
        best_l = max(res, key=lambda key: res[key]['mn'])
        best_c = max(res, key=lambda key: res[key]['mc'])
        if res[best_l]['mn'] > 0.5:
            identified = res[best_l]['mn']
            if best_c != 0:
                if res[best_c] != identified:
                    identified = res[best_c]
        if identified != '':
            delegate = fm.match_candidates(heer=best_l.name)
            identified = delegate
        else: 
            identified = None
        if identified:
            
            name = delegate.proposed_delegate
            m_id = delegate.id
        else:
            name = ''
            m_id = 0
        start = ob.item.find(i)
        end = start + len(i)
        ob.set_span(span=(start, end),pattern=i, clas='deputy', delegate_name=name,
                    delegate_id=m_id)
#         defo[i]={'mn':(best_l,res[best_l]['mn']),
#                  'co':(best_c,res[best_c]['mc'])}
# defo

In [205]:
def get_mn(k, defo=defo):
    defo = dict(defo)
    v = defo[k]['mn'][1]
    v = float(v)
    return v
    
#         mn = dict(defo[k].get('mn').value
for k in defo:
    v = get_mn(k)
    if v < 0.5:
        res = iterative_search(name=k, year=1728, df=abbreviated_delegates)
        res

In [206]:
s = moregentlemen[8]
print(s.variants)
for v in s.variants:
    print(score_levenshtein_distance_ratio('an Schwartzenberg', s.name))
    #list(map(lambda x: score_levenshtein_distance_ratio('an Schwartzenberg', x), s.variants))

{'general': [V an S chwart zenbergh., Van Schwartzenbergh., Sehwartzenbergh. P, Han Schwartzenbergh., Van Sebwartzenbergh., an Schwartzenberg., Pan Schwartzenbergh., Van Schwartzenhergh., Van Sehwartzenbergh. P, Jan Schwartzenbergb., Van Schwartzenberg., Han Schwartzenberg., Van Schwartzenbergh.  P, Zan Schwartzenbergh., Van Schwartzenbergh. P, an Schwartzenbergh. P, an Schwart zenhergh., Schwartzenberg, Schwartzenberdh, zenbargh, zenborgl, Sebwartzenbergh, Schwartzenbersk, zenhorgh, Cattenboreh, Cartenborgh, Schwartzenberth, Cattenhorgh, Schwartzenberch, Cattenbergh, Schwattzenbergh, Schwartzenbergb, attenborgh, Schwartzenbereh, Schwartzenberghb, Zzenhorgh, Cáttenborch, Cattcnborgh, zenbergh, zenberghh, Cattenborgh, Cattenaorgh, tzenborghn, Caitenborgh, Sehwartzenbergh, Schwartzenbersh, Cattcnbergh, Schwartzènherch, Cattenborsh, Schwartvenbergh, Schwartenbergh, Cattenboroh, chwartzenbergh, Sehwartzenbergb, Schwartzenbersb, zenborah, tenboreh, Cattenbor, Zenborgh, zenboreh, Scbwartzenb

In [ ]:
v = s.variants['general'][0]
v.form

In [ ]:
res

In [120]:
span.pattern

'Tamminga.'

In [121]:
# span.set_delegate(delegate_id=10, 
#                               delegate_name='blop', 
#                               delegate_score=1.0)
span.delegate_id

17356

In [122]:
all_matched.keys()

dict_keys([13046, 16377, 18838, 16901, 17355, 17753, 16423, 15207, 12367, 16446, 13477, 17094, 18599, 17356, 17379, 16160, 17397, 16465, 15226, 19094, 12332, 19119, 13100, 2798, 12744, 19297, 14821, 15268, 13492, 13479, 19114, 18570, 17505, 16400, 14074, 17702, 19627, 19130, 19518, 15059, 16447, 12358, 13458, 17837, 17942, 19181, 14865, 19871, 19778, 17563, 17268, 16303, 453, 12537, 17635, 17927, 19123, 14047, 17340, 16492, 14079, 13401, 13911, 16491, 14573, 16479, 17090, 1451, 15265, 13384, 16404, 14697])

In [123]:
previously_matched

,Unnamed: 0,name,match_keyword,levenshtein_distance,proposed_delegate,id,score,probably_junk
0,7,Singendonck,Singendonck,1.000000,Singendonck,19094,1.0,False
1,8,Torck,Torck,1.000000,Torck,19114,1.0,False
2,9,Umbgroeven,Umbgrove,0.888889,Umbgrove,17268,1.0,False
3,11,Boetzelaar,Boetzelaar,1.000000,van Boetzelaar,17501,1.0,False
4,12,Steyn,Steyn,1.000000,Steyn,14009,1.0,False
5,13,Marseveen,Maarseveen,0.900000,Huydecoper van Maarseveen,13642,0.5,False
6,14,Zuylen,Zuylen,1.000000,van Zuylen van Nyvelt,18913,1.0,False
7,15,Nyvelt,Nyvelt,1.000000,van Zuylen van Nyvelt,18913,0.9,False
8,16,Boon,Boon,1.000000,Boon,15168,0.9,False
9,17,Velters,Velters,1.000000,Velters,17094,1.0,False


### Some Evaluations (fwiw)

In [124]:
pd.DataFrame(previously_matched.aggregate("mean")[['levenshtein_distance', 'score']])

,0
levenshtein_distance,0.807220
score,0.780645


In [125]:
previously_matched.score.value_counts()

1.0    19
0.0     5
0.9     3
0.6     2
0.8     1
0.5     1
Name: score, dtype: int64

In [126]:
previously_matched.levenshtein_distance.round(2).value_counts().sort_values('index', ascending=False)

1.00    19
0.00     5
0.89     3
0.80     2
0.86     1
0.90     1
Name: levenshtein_distance, dtype: int64

In [127]:
for s in ob.spans:
    print(s, s.idnr, ob.item[s.begin:s.end], s.get_delegate())

fragment type preamble: 0-30 1360 Veneris den 31. December 1728. {'id': 0, 'name': '', 'score': 0}
fragment type deputy: 0-0 8537  {'id': 0, 'name': '', 'score': 0}
fragment type deputy: 0-0 8539  {'id': 0, 'name': '', 'score': 0}
fragment type deputy: 0-0 8540  {'id': 0, 'name': '', 'score': 0}
fragment type deputy: 0-0 8541  {'id': 0, 'name': '', 'score': 0}
fragment type deputy: 0-0 8542  {'id': 0, 'name': '', 'score': 0}
fragment type deputy: 0-0 8543  {'id': 0, 'name': '', 'score': 0}
fragment type deputy: 0-0 8544  {'id': 0, 'name': '', 'score': 0}
fragment type deputy: 0-0 8545  {'id': 0, 'name': '', 'score': 0}
fragment type deputy: 0-0 8546  {'id': 0, 'name': '', 'score': 0}
fragment type deputy: 0-0 8547  {'id': 0, 'name': '', 'score': 0}
fragment type deputy: 0-0 8548  {'id': 0, 'name': '', 'score': 0}
fragment type deputy: 0-0 8549  {'id': 0, 'name': '', 'score': 0}
fragment type deputy: 0-0 8550  {'id': 0, 'name': '', 'score': 0}
fragment type deputy: 0-0 8551  {'id': 0, '

## a better searcher and identifier.

The problem: 
We have two ways of identifying name patterns now.

- the first uses pandas based method to identify strings against a database of known delegates and then pulls in some of the external context. This works pretty well for initial identification, but not all names are recognized very well. There is especially a problem by names that have been badly garbled by the text recognition, which is now the case with the OCR, but it will probably not get better if we start using HTR and look up entities in the text. Another issue is that this method has no memory of earlier work, both in the sense that it will not know if earlier matches were found or earlier proposals were marked as false. 
- the fuzzysearcher is very good for spotting name patterns in text as it includes approximate text matching and a number of variants. But it knows nothing about identification and there is no easy way of associating patterns with entities.
- fuzzysearcher is also able to cluster alike word patterns and recluster heterogeneous lists. However, this does not per se bring us closer to identification
- the only way to include previously selected variants is in the fuzzy searcher. Which is useful, but the main problem remains associating this with external knowledge.

This leads to the following requirements:
- a way of associating patterns with identities. But how. Earlier I put the result of a Fuzzysearch pattern clustering in a dataframe, but this included heterogeneous patterns. One possibility is to revise candidates and recategorize weak candidates. The goals is to incrementally improve the patterns associated with a name. This is especially useful if we reuse the objects over many years.
- another idea is to check for internal consistency of the results and try and find inconsistent results. This is especially useful for the presidents, as their number is even more limited than the delegates. The problem is overfitting in which unidentified names get associated with identified names and also new or infrequent delegates may drop out. SO there is a need for manual check.
- another tool may be to make a specialized confusion matrix for delegates

We start with this latest
no we don't. Most of this is already in place. The main problem is that we cannot access the it from 'both' sides, so that we have to rematch all over again. Which is what we do not want, because it slows down the process *and* it introduces mistakes all over again. What we need is memory:
- of previous matches
- of previous associations
- of previous wrong associations
and perhaps more

    

In [128]:
def get_delegates_from_spans(ob):
    result = {}
    for s in ob.spans:
        result['delegate'] = s.get_delegate()
        result['pattern'] = s.pattern
    return result

In [129]:
previously_matched.columns

Index(['Unnamed: 0', 'name', 'match_keyword', 'levenshtein_distance',
       'proposed_delegate', 'id', 'score', 'probably_junk'],
      dtype='object')

In [150]:
unclassified = []
nwheren = []
clean_gentlemen = []
for g in moregentlemen:
    matches = defaultdict(list)
    matchkw = g.matchkw
    matchkw = matchkw.strip()
    nwmatches = {}
    if matchkw == '':
        unclassified = g.variants['general'] # we go to the next instance and look into this later
        continue
    for v in g.variants['general']:
        matches[v.match].append(v)
        for key in matches.keys():
            name = g.name
            nwmatches[matchkw] = []
            s = score_levenshtein_distance_ratio(key, matchkw)
            if s > 0.7:
                nwmatches[matchkw].extend(matches[key])
            else:
                if not re.search('\w+', key):
                    #this is an empty key
                    meanscore = mean([score_ngram_overlap_ratio(kw.form, matchkw, 1) for kw in matches[key]])
                    if meanscore > 0.8:
                         nwmatches[matchkw].extend(matches[key])
        #            print('meanscore: ', key, meanscore)

                # is there no strange 'Van' variant in this?
                elif len(key)<5:
                    s2 = score_ngram_overlap_ratio(key, 'van', 1)
                    if s2 > 0.8:
                        meanscore = mean([score_levenshtein_distance_ratio(kw.form, matchkw, 1) for kw in matches[key]])
                        nwmatches[matchkw].extend(matches[key])
                        #print('meanscore: ', key, meanscore)
                    else: 
                        unclassified.extend(matches[key])
                else:
                    meanheerscore = mean([score_levenshtein_distance_ratio(kw.form, matchkw) for kw in matches[key]])
                    meankeyscore = mean([score_levenshtein_distance_ratio(kw.form, key) for kw in matches[key]])
                    meanheergramscore = mean([score_ngram_overlap_ratio(kw.form, matchkw, 2) for kw in matches[key]])
                    meankeygramscore = mean([score_ngram_overlap_ratio(kw.form, key, 2) for kw in matches[key]])
                    if meankeyscore>meanheerscore and meankeygramscore>meanheergramscore:
                        #print(key, meankeyscore-meanheerscore, meankeygramscore-meanheergramscore)
                        nwres = iterative_search(key, year=day, df=abbreviated_delegates)
                        rec = {'id': nwres.id.iat[0],
                               'm_kw': key,
                               'name': nwres.name.iat[0],
                               'variants': [(v.form, v.match, v.score) for v in matches[key]]}
                        nwheer = MatchHeer(rec=rec)
                        nwheren.append(nwheer)
    g.variants['new']=nwmatches
        
                
            

In [151]:
allvariants = Counter()
for g in moregentlemen:
    allvariants.update(g.variants['general'])
allvariants.most_common()

[(Bofeh, 18),
 (Taais, 16),
 (Taas, 15),
 (Zaats, 10),
 (7aats, 9),
 (aats, 7),
 (cock, 7),
 (Taals, 6),
 (Eelke, 6),
 (Pall, 6),
 (Danck-, 5),
 (Deck, 5),
 (Bers, 4),
 (Catten, 4),
 (Boftl, 3),
 (vats, 2),
 (Aan, 2),
 (ooch, 2),
 (Dack-, 2),
 (haaren, 2),
 (voor, 2),
 (Van Glinstra. P, 1),
 (Zan Giinftra., 1),
 (2an Glinftra., 1),
 (Jan Glinfra., 1),
 (Zan Glinftra., 1),
 (an Gliniira., 1),
 (Han Glinftra., 1),
 (Jan Glinftra, 1),
 (an Glinjtra., 1),
 (Van Glinstra., 1),
 (Van Glinftra., 1),
 (Van Glinflra., 1),
 (an Glinftra., 1),
 (Glnfra, 1),
 (Goflina, 1),
 (Goflinga, 1),
 (Goslinca, 1),
 (Glinsra, 1),
 (Glinfka, 1),
 (Glintra, 1),
 (Glnftra, 1),
 (Goslinga, 1),
 (Glnhra, 1),
 (CGlenstra, 1),
 (Clinftra, 1),
 (Ginftra, 1),
 (Glinfira, 1),
 (Glnlra, 1),
 (Glinfra, 1),
 (Glina, 1),
 (Glinftra, 1),
 (Clinfra, 1),
 (Glinstra, 1),
 (Gofliaga, 1),
 (Clinsra, 1),
 (Gilnftra, 1),
 (Glnlira, 1),
 (Gollinva, 1),
 (Glinhra, 1),
 (Gollinea, 1),
 (CGlinstra, 1),
 (Glînftra, 1),
 (Clinstra, 1),

ok, so we need something do discriminated them. Perhaps the time of their actual delegate appointments?
another idea is to try and match names more closely. But we do not implement that here (as I do not think there is much to gain here as yet)

In [153]:
clbm = DisambiguateDelegate()
clbm.closer_best_match(name='Aerssen', date="1628")

{'id': 38168,
 'exact': False,
 'row':   functienaam                                            college    regent  \
 1    griffier  Staten-Generaal der Verenigde Nederlanden (158...  Aerssens   
 
       id van van_als_bekend tot tot_als_bekend  persoon_id  functie_id  ...  \
 1  38168 NaT     28-09-1584 NaT        03-1627       17183         121  ...   
 
       old_lokaal old_provinciaal    old_regio    old_stand  \
 1  republiek_nan     republiek_0  republiek_0  republiek_0   
 
   old_vertegenwoordigend  periode       van_p       tot_p  \
 1            republiek_0        0  1584-09-28  1627-03-01   
 
                                             p_interv    daycompare  
 1  PeriodIndex(['1584-09-28', '1584-09-29', '1584...  <306 * Days>  
 
 [1 rows x 37 columns]}

# Consistency Checks
## Not finished

In [154]:
unmt = [searchobs[so].matched_text.get_unmatched_text() for so in searchobs]

In [155]:
wrds = [k[0].form for k in allvariants.most_common() if k[0] not in ['',None]]
fks.find_close_distance_keywords(wrds)


defaultdict(list,
            {'Bofeh': ['Bofih', 'Bofch', 'Bofth'],
             'Bofih': ['Bofeh', 'Bofib', 'Bofch', 'Bofth'],
             'Bofch': ['Bofeh', 'borch', 'Bofih', 'Bosch', 'Bofth'],
             'Bofth': ['Bofeh', 'Boftl', 'Bofih', 'Bofch'],
             'Taais': ['Taas', 'Taals', 'Taats'],
             'Taas': ['Taais', 'Taals', 'Taats'],
             'Taals': ['Taais', 'Taas', 'Taats'],
             'Taats': ['Taais',
              'Taas',
              'Zaats',
              '7aats',
              'aats',
              'Taals',
              'Tuats',
              'Táats',
              'Tauts',
              'Tagts',
              'Taáts'],
             'Zaats': ['7aats', 'aats', 'Taats'],
             '7aats': ['Zaats', 'aats', 'Taats'],
             'aats': ['Zaats', '7aats', 'vats', 'Taats'],
             'vats': ['aats', 'Vas', 'vas'],
             'cock': ['oock', 'Cocq', 'Coca', 'Cocg', 'Coco'],
             'oock': ['cock', 'ooch', 'Roock'],
             'Coc

In [156]:
for nwh in nwheren:
    if nwh.heerid not in all_matched.keys():
        print(nwh.heerid, nwh.name)

13050 van Goslinga
13050 van Goslinga
13050 van Goslinga
13050 van Goslinga
13050 van Goslinga
13050 van Goslinga
13050 van Goslinga
13050 van Goslinga
13050 van Goslinga
13050 van Goslinga
13050 van Goslinga
13050 van Goslinga
13050 van Goslinga
13050 van Goslinga
13050 van Goslinga
13050 van Goslinga
13050 van Goslinga
13050 van Goslinga
13050 van Goslinga
13050 van Goslinga
13050 van Goslinga
13050 van Goslinga
13050 van Goslinga
13050 van Goslinga
13050 van Goslinga
13050 van Goslinga
13050 van Goslinga
13050 van Goslinga
13050 van Goslinga
13050 van Goslinga
17862 van Welderen
17862 van Welderen
17862 van Welderen
17862 van Welderen
17862 van Welderen
17862 van Welderen
17862 van Welderen
17862 van Welderen
17862 van Welderen
17862 van Welderen
17862 van Welderen
17862 van Welderen
17862 van Welderen
16495 Westenberg
16495 Westenberg
16495 Westenberg
16495 Westenberg
16495 Westenberg
16495 Westenberg
16495 Westenberg
16495 Westenberg
16495 Westenberg
16495 Westenberg
16495 Westenb

In [157]:
def joinheer(h1, h2):
    if not h1.heerid == h2.heerid:
        raise ValueError('ids not equal')
    forms1 = set([v.form for form in h1.variants])
    forms2 = set([v.form for form in h2.variants])
    difference = forms2.difference(forms1)
    
    for form in difference:
        v1 = [v for v in h2.variants['general'] if v.form==form][0]
        h1.variants['general'].append(v1)
    return h1

In [158]:
nnwheren = []


In [159]:
idvals = set([n.heerid for n in nwheren])
clusterednwh = defaultdict(list)
for heerid in idvals:
    clusterednwh[heerid] = [nwh for nwh in nwheren if nwh.heerid == heerid]
    

In [160]:
from itertools import groupby
ds = {}
sh = sorted(nwheren, key=lambda x: x.heerid)
for k, g in groupby(sh, key=lambda x: x.heerid):
    ds[k] = list(g)
for k in ds:
    h1 = ds[k][0]
    for h2 in ds[k][1:]:
        h1 = joinheer(h1,h2)
    nnwheren.append(h1)

In [161]:
nnwheren[2].variants

{'general': [churman]}

In [162]:
unclassified

[voorle-,
 voorlede,
 Taais,
 Taais,
 Taas,
 Taais,
 Taas,
 Taais,
 Taas,
 Taais,
 Taas,
 Taais,
 Taas,
 Taais,
 Taas,
 Zaats,
 Taais,
 Taas,
 Zaats,
 7aats,
 Taais,
 Taas,
 Zaats,
 7aats,
 Taais,
 Taas,
 Zaats,
 7aats,
 aats,
 Taais,
 Taas,
 Zaats,
 7aats,
 aats,
 Taals,
 Taais,
 Taas,
 Zaats,
 7aats,
 aats,
 Taals,
 Taais,
 Taas,
 Zaats,
 7aats,
 aats,
 Taals,
 Taais,
 Taas,
 Zaats,
 7aats,
 aats,
 Taals,
 Taais,
 Taas,
 Zaats,
 7aats,
 aats,
 vats,
 Taals,
 Aan,
 Bers,
 Bers,
 Bers,
 Bofeh,
 Bofeh,
 Bofeh,
 Bofeh,
 Bofeh,
 Bofeh,
 Bofeh,
 Bofeh,
 Bofeh,
 Bofeh,
 Bofeh,
 Bofeh,
 Bofeh,
 Bofeh,
 Bofeh,
 Bofeh,
 Boftl,
 Bofeh,
 Boftl,
 Catten,
 Catten,
 Catten,
 cock,
 cock,
 cock,
 cock,
 cock,
 cock,
 ooch,
 Danck-,
 Danck-,
 Danck-,
 Danck-,
 Dack-,
 Deck,
 Deck,
 Deck,
 Deck,
 Eelke,
 Eelke,
 Eelke,
 Eelke,
 Eelke,
 haaren,
 voor,
 Pall,
 Pall,
 Pall,
 Pall,
 Pall]

In [163]:
unob = unclassified[0]
unob.heer


'Vegelin van Claerbergen'

In [164]:
c = Counter(unclassified)
c.most_common()

[(Bofeh, 17),
 (Taais, 15),
 (Taas, 14),
 (Zaats, 9),
 (7aats, 8),
 (aats, 6),
 (cock, 6),
 (Taals, 5),
 (Eelke, 5),
 (Pall, 5),
 (Danck-, 4),
 (Deck, 4),
 (Bers, 3),
 (Catten, 3),
 (Boftl, 2),
 (voorle-, 1),
 (voorlede, 1),
 (vats, 1),
 (Aan, 1),
 (ooch, 1),
 (Dack-, 1),
 (haaren, 1),
 (voor, 1)]

In [165]:
unclas = fks.find_close_distance_keywords([v.form for v in unclassified])
G_unclas = nx.Graph()
d_nodes = sorted(unclas)
for node in d_nodes:
    attached_nodes = unclas[node]
    G_unclas.add_node(node)
    for nod in attached_nodes:
        G_unclas.add_edge(node, nod)
connected_unclas = list(nx.connected_components(G_unclas))


In [166]:
connected_unclas

[{'7aats', 'Zaats', 'aats', 'vats'},
 {'Bers'},
 {'Bofeh'},
 {'Boftl'},
 {'Catten'},
 {'Dack-', 'Danck-'},
 {'Deck'},
 {'Eelke'},
 {'Pall'},
 {'Taais', 'Taals', 'Taas'},
 {'cock'},
 {'voorle-', 'voorlede'}]

In [167]:
iterative_search('Ten Brincke', year=1728, df=abbreviated_delegates)


,id,name,geboortejaar,sterfjaar,age_at_repr,colleges,functions,period,sg,was_gedeputeerde,p_interval,h_life,score
4913,16400,ten Brink,1690-01-01,1757-09-10,36.0,"Raad van Deventer, Staten-Generaal der Verenig...","lid, ordinaris gedeputeerde, gecommitteerde","PeriodIndex(['1716-01-01', '1716-01-02', '1716...",True,True,"[1716, 1757]","[1690, 1757]",0.8
6252,17925,van Borck,NaT,1720-03-22,NaN,"Raad van Groningen, Gedeputeerde Staten van Gr...","lid, gecommitteerde, ordinaris gedeputeerde","PeriodIndex(['1695-01-01', '1695-01-02', '1695...",True,True,"[1695, 1720]","[1651, 1720]",0.8


In [168]:
iterative_search('Eck', year=1728, df=abbreviated_delegates)

,id,name,geboortejaar,sterfjaar,age_at_repr,colleges,functions,period,sg,was_gedeputeerde,p_interval,h_life,score
7307,19133,van Eck tot Nergena,1697-01-01,1760-12-31,29.0,"Ridderschap van het Kwartier Veluwe, Staten-Ge...","lid, ordinaris gedeputeerde","PeriodIndex(['1720-01-01', '1720-01-02', '1720...",True,True,"[1720, 1760]","[1697, 1760]",1.0
7834,19858,van Eck,1682-01-01,1736-11-28,44.0,"Hof van Gelre en Zutphen (1588-1795), Staten-G...","griffier, raadsheer, ordinaris gedeputeerde","PeriodIndex(['1708-10-25', '1708-10-26', '1708...",True,True,"[1708, 1738]","[1682, 1736]",1.0


In [141]:
for x in connected_unclas:
    for i in x:
        if not re.search('[0-9]+',i):
            print (i)
#         ir = [v for v in unclassified if v.form == i]
#         for vir in ir:
#             print (vir.form, vir.heerid, '\n\n')
            result = herensearcher.find_candidates(i)
            if len(result) > 0:
                eindresult = max(result, key=lambda keyword: keyword['levenshtein_distance'])
                

Zaats
aats
vats
Catten
Danck-
Dack-
Deck
Eelke
Pall
Taals
Taais
Taas
ooch
voor
voorle-
voorlede


# Placeholder

In [173]:
def span_checker(ob):
    """prints all spans from a searchobject"""
    spans = ob.matched_text.spans
    for span in spans:
        txt = ob.matched_text.item[span.begin:span.end]
        span.set_pattern(txt)

        #kand = matchsearch.find_candidates(txt)[0]
        msk = framed_gtlm.vs.apply(lambda x: levenst_vals(x, txt))
        mres = framed_gtlm.loc[msk==msk.max()]
        if len(mres)>0:

    #         setattr(span, 'delegate_id', mres.uuid.iat[0])
    #         setattr(span, 'delegate_name', mres.name.iat[0])
        #print(kand, all_matched[kand])
            span.set_delegate(delegate_id=mres.ref_id.iat[0], delegate_name=mres.name.iat[0], delegate_score=msk.max())
        print(span.type, span.idnr, txt, span.delegate_id, span.delegate_name, span.delegate_score)
